# THIS IS JUST TO SEE IF EVERYTHING WORKS

Since I don't want to upload every script manually, all the scripts content will be initialized here.

Dependencies

In [1]:
!apt install ffmpeg
!pip install ultralytics

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.5/756.5 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB

In [26]:
from IPython.display import clear_output

!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz
clear_output()
!7z e /usr/local/bin/ffmpeg.tar
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
%cd /content/
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-70.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
clear_output()
!ffmpeg -version

ffmpeg version N-115141-g4e069ba80a-20240508 Copyright (c) 2000-2024 the FFmpeg developers
built with gcc 13.2.0 (crosstool-NG 1.26.0.65_ecc5e41)
configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-ffbuild-linux-gnu- --arch=x86_64 --target-os=linux --enable-gpl --enable-version3 --disable-debug --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-openssl --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --enable-libpulse --enable-libvmaf --enable-libxcb --enable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libaribcaption --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopu

### Frame Extractor

In [32]:
import math
import os
import subprocess
import sys
import cv2 as cv


import os
import subprocess

def extract_frames(video_path, output_path, fps):
    """
    Extract frames from a video file and save them as individual images.

    Args:
        video_path (str): The path to the video file.
        output_path (str): The path to the output folder where the frames will be saved.
        fps (int): The desired frames per second for the extracted frames.

    Returns:
        None
    """
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    video_name = os.path.basename(video_path)
    # remove the file extension
    video_name = os.path.splitext(video_name)[0]
    output_folder = os.path.join(output_path, video_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    print(f"Extracting frames to {output_folder}")

    # Handle the fact that the directory may contain whitespaces
    video_path = f'"{video_path}"'
    output_folder = f'"{output_folder}"'

    command = f"ffmpeg -i {video_path} -vf fps={fps} {output_folder}/%d.jpg"
    subprocess.call(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


def fps_count(seconds, factor):
    """
    Calculate the number of frames per second based on the number of seconds and a factor.

    Args:
        seconds (int): The number of seconds in the video.
        factor (float): The factor to use in the calculation.
    """
    return math.floor(math.log(seconds, 10) * factor)



# json for the input
def frame_extraction(input_json, verbose=False):
    """
    Extract frames from a video file and save them as individual images.

    Args:
        video_path (str): The path to the video file.
        output_path (str): The path to the output folder where the frames will be saved.
        factor (float): The factor to use in the calculation of the frames per second.
        verbose (bool): Whether to print additional information.
    """
    video_path = input_json["video_path"]
    output_path = input_json["output_path"]
    factor = input_json["factor"]

    if not os.path.exists(video_path):
        print(f"Video path: {video_path}")
        print("Video file does not exist")
        sys.exit(1)

    video = cv.VideoCapture(video_path)

    fps = video.get(cv.CAP_PROP_FPS)
    frame_count = video.get(cv.CAP_PROP_FRAME_COUNT)

    print(fps)
    sec = (int)(frame_count / fps)

    frames = fps_count(sec, factor)
    fps = math.floor(frames / sec * 100) / 100  # floor to two decimal places

    if verbose:
        print(
            f"Total extracted frames: {frames}\nSeconds video duration: {sec}\nFPS: {fps}"
        )
    extract_frames(video_path, output_path, fps)
    if verbose:
        print("Frames extracted successfully")

    output_json = {
        "output_path": os.path.join(output_path, os.path.splitext(os.path.basename(video_path))[0]),
        "frame_count": frames,
        "video_id": os.path.basename(video_path),
    }

    return output_json

### YOLOv8

In [43]:
from ultralytics import YOLO
import os
import json
import json

model = YOLO("yolov8n.pt")  # load an official model

def get_yolo_labels(input_json):

    images = os.listdir(input_json["output_path"])
    # sort images by number
    images.sort(key=lambda x: int(x.split(".")[0]))

    res = model(input_json["output_path"])

    result = []
    for ix, r in enumerate(res):
        els = set()

        # If there are no boxes, skip
        try:
            for c in r.boxes.cls:
                els.add(model.names[int(c)])
        except:
            pass

        result.append({"frame": ix, "labels": list(els)})

    json_result = json.dumps(result)

    return json_result

Michele's code for Framework selection

In [1]:
pass

### Main code

In [44]:
# Example of usage
import os
import json

# 1. Get the video
dir_path = "/content"
video_path = os.path.join(dir_path, "room.mp4")

# 2. Extract frames
input_json = {
    "video_path": video_path,
    "output_path": os.path.join(dir_path, "output"),
    "factor": 10
}

print(input_json)
extraction = frame_extraction(input_json, verbose=False)
print(f"The extraction is:")
print(json.dumps(extraction, indent=4))

# 3. Get frames labels using YOLO
labels = get_yolo_labels(extraction)

print(f"Labels for the frames:")
for l in json.loads(labels):
    print(l)

{'video_path': '/content/room.mp4', 'output_path': '/content/output', 'factor': 10}
59.63234331670284
Extracting frames to /content/output/room
The extraction is:
{
    "output_path": "/content/output/room",
    "frame_count": 10,
    "video_id": "room.mp4"
}

image 1/11 /content/output/room/1.jpg: 640x384 1 suitcase, 11.9ms
image 2/11 /content/output/room/10.jpg: 640x384 1 person, 1 cup, 2 chairs, 2 laptops, 2 mouses, 2 keyboards, 9.3ms
image 3/11 /content/output/room/11.jpg: 640x384 1 person, 1 cup, 2 chairs, 1 dining table, 1 laptop, 1 mouse, 3 keyboards, 6.8ms
image 4/11 /content/output/room/2.jpg: 640x384 (no detections), 6.8ms
image 5/11 /content/output/room/3.jpg: 640x384 1 backpack, 2 suitcases, 6.7ms
image 6/11 /content/output/room/4.jpg: 640x384 (no detections), 6.6ms
image 7/11 /content/output/room/5.jpg: 640x384 (no detections), 10.0ms
image 8/11 /content/output/room/6.jpg: 640x384 1 bed, 6.9ms
image 9/11 /content/output/room/7.jpg: 640x384 1 chair, 1 bed, 6.9ms
image 10/11

### Moondream

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import torch

model_id = "vikhyatk/moondream2"
revision = "2024-04-02"
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)
torch.cuda.is_available()

image = Image.open("Scripts/output/cat/chrome_idanP15BYp.png")
enc_image = model.encode_image(image)
print(model.answer_question(enc_image, "Describe this image considering it will be used as input to a sound generation model", tokenizer))
